<a href="https://colab.research.google.com/github/ArthurFDLR/GANightSky/blob/main/styleGAN2-ADA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# StyleGan2-ADA in Google Colab

StyleGAN2-ADA only work with Tensorflow 1. Run the next cell before anything else to make sure we’re using TF1 and not TF2.

In [ ]:
%tensorflow_version 1.x
!nvidia-smi

## Install [StyleGAN2-ADA Repository](https://github.com/NVlabs/stylegan2-ada) in your Google Drive

First, mount your Drive to the Colab notebook: 

In [ ]:
from google.colab import drive
from pathlib import Path

content_path = Path('/').absolute() / 'content'
drive_path = content_path / 'drive'
drive.mount(str(drive_path))

Next, run this cell. If you’ve already installed the repository, it will skip the installation process and only check for updates. If you haven’t installed it, it will install all the necessary files. Beside, **in**, **out**, **datasets** and **training** folders are generated for data storage. Everything will be available on your Google Drive in the folder **StyleGAN2-ADA** even after closing this Notebook.

In [ ]:
stylegan2_repo_url  = 'https://github.com/ArthurFDLR/stylegan2-ada' # Original repository: https://github.com/NVlabs/stylegan2-ada
project_path        = drive_path / 'MyDrive' / 'StyleGAN2-ADA'
stylegan2_repo_path = project_path / 'stylegan2-ada'

# Create project folder if inexistant
if not project_path.is_dir():
    %mkdir "{project_path}"
%cd "{project_path}"

for dir in ['in', 'out', 'datasets', 'training']:
    if not (project_path / dir).is_dir():
        %mkdir {dir}
if not (project_path / 'datasets' / 'source').is_dir():
    %mkdir "{project_path / 'datasets' / 'source'}"

# Download StyleGAN2-ada
!git config --global user.name "ArthurFDLR"
!git config --global user.email "arthfind@gmail.com"
if stylegan2_repo_path.is_dir():
    !git -C "{stylegan2_repo_path}" fetch origin
    !git -C "{stylegan2_repo_path}" checkout origin/main -- *.py
else:
    !git clone {stylegan2_repo_url}

## Generate image from pre-trained model

You can now generate images using pre-trained networks. NVLab published some of their network pickles on [their website](https://nvlabs-fi-cdn.nvidia.com/stylegan2-ada/pretrained/). You can also use your own model once it is trained (we will come to model training in a few cells).

In [ ]:
%pip install opensimplex
!python "{stylegan2_repo_path / 'generate.py'}" generate-images --help 

In [ ]:
from numpy import random
seed_init = random.randint(10000)
nbr_images = 2

!python "{stylegan2_repo_path / 'generate.py'}" generate-images \
    --outdir="{project_path / 'out'}" --trunc=0.7 \
    --seeds={seed_init}-{seed_init+nbr_images-1} --create-grid \
    --network=https://nvlabs-fi-cdn.nvidia.com/stylegan2-ada/pretrained/ffhq.pkl

## Latent space exploration

In [ ]:
%pip install opensimplex
!python "{stylegan2_repo_path / 'generate.py'}" generate-latent-walk --help 

In [ ]:
from numpy import random
walk_types = ['line', 'sphere', 'noiseloop', 'circularloop']
latent_walk_path = project_path / 'out' / 'latent_walk'
if not latent_walk_path.is_dir():
    %mkdir "{latent_walk_path}"

explored_network = '/content/drive/MyDrive/StyleGAN2-ADA/training/NightSky/00010-tfr-mirror-auto1-bgc-resumecustom/network-snapshot-000256.pkl'
seeds = [519, 1343, 1431, 1535, 1599, 1764, 519] #[random.randint(10000) for  i in range(20)] 
print("Seeds:", seeds)
!python "{stylegan2_repo_path / 'generate.py'}" generate-latent-walk --network="{explored_network}" \
    --outdir="{latent_walk_path}" --trunc=0.7 --walk-type="{walk_types[2]}" \
    --seeds={','.join(map(str, seeds))} --frames {len(seeds)*20}

## Train a custom model

In [ ]:
dataset_name = 'NightSky'
datasets_path = project_path / 'datasets' / dataset_name
training_path = project_path / 'training' / dataset_name

if not datasets_path.is_dir():
    print("Need to convert dataset to .tfrecords")
else:
    print('Ready for training!')

Large dataset might exceed Google Drive quota after a few training batches. Indeed, StyleGAN2 download datasets multiple times during training. You might have to import your dataset in the local storage session. However, large files cannot be copy/paste from Drive *(Input/Output error)*. 

To download your zipped dataset from your drive and unzip it in local machine, run this cell.

In [ ]:
local_dataset_path = content_path / 'dataset'
if not local_dataset_path.is_dir():
    %mkdir "{local_dataset_path}"
    %cp -a "{project_path / 'datasets' / 'source' / (dataset_name + '.zip')}" "{local_dataset_path}"
    print("Zip file imported")
else:
    print('Allready imported')

import zipfile
with zipfile.ZipFile(str(local_dataset_path / (dataset_name + '.zip')), 'r') as zip_ref:
    zip_ref.extractall(str(local_dataset_path))

### Convert dataset to .tfrecords

**Note: You only need to do this once per dataset. If you have already run this and are returning to conntinue training, skip these cells.**

Next we need to convert our image dataset to a format that StyleGAN2-ADA can read from. There are two options here. You can upload your dataset directly to Colab (as a zipped file), or you can upload it to Drive directly and read it from there.

Now that your image dataset is uploaded, we need to convert it to the `.tfrecords` format.

Depending on the resolution of your images and how many you have, this can take a while.

In [ ]:
local_images_path = local_dataset_path / 'images'
local_dataset_path /= 'tfr'

if (local_dataset_path).is_dir():
    print('\N{Heavy Exclamation Mark Symbol} Dataset already created \N{Heavy Exclamation Mark Symbol}')
    print('Delete current dataset folder ({}) to regenerate tfrecords.'.format(local_dataset_path))
else:
    %mkdir "{local_dataset_path}"
    !python "{stylegan2_repo_path / 'dataset_tool.py'}" create_from_images \
        "{local_dataset_path}" "{local_images_path}"

### Launch training

There are numerous arguments to tune the training of your model. To obtain nice results, you will certainly have to experiment. Here are the most popular options:


*   *mirror:* Should the images be mirrored vertically?
*   *mirrory:* Should the images be mirrored horizontally?
*   *snap:* How often should the model generate image samples and a network pickle (.pkl file)?
*   *resume:* Network pickle to resume training from?

To see all the options, run the following cell.

In [ ]:
!python "{stylegan2_repo_path / 'train.py'}" --help

In [ ]:
if not training_path.is_dir():
    %mkdir "{training_path}"

#how often should the model generate samples and a .pkl file
snapshot_count = 2
#should the images be mirrored left to right?
mirrored = True
#should the images be mirrored top to bottom?
mirroredY = False
#metrics? 
metric_list = None
#augments
augs = 'bgc'

#resume_from = 'ffhq1024'
resume_from = '/content/drive/MyDrive/StyleGAN2-ADA/training/NightSky/00009-tfr-mirror-auto1-bgc-resumecustom/network-snapshot-000112.pkl'

!python "{stylegan2_repo_path / 'train.py'}" --outdir="{training_path}" \
    --data="{local_dataset_path}" --resume="{resume_from}" \
    --snap={snapshot_count} --augpipe={augs} \
    --mirror={mirrored} --metrics={metric_list} #--dry-run #--mirrory={mirroredY}

## While you wait ...

... learn more about StyleGAN2-ADA and Generative Adversarial Networks in general:

*   [Karras, Tero, et al. _Analyzing and Improving the Image Quality of StyleGAN._ CVPR 2020.](https://arxiv.org/pdf/2006.06676.pdf): Paper published for the release of StyleGAN2-ADA.
*   [Official implementation of StyleGAN2-ADA](https://github.com/NVlabs/stylegan2-ada)
